In [655]:
import xml.etree.ElementTree as et
import dateutil.parser as parser
import pandas as pd
import datetime
from timezone import get_time_by_local


# tree = et.parse('xmls/RS_Via-3.xml')
# root = tree.getroot()


class Query:
    def __init__(self, xml):
        self.xml = xml

    def get_xml_root(self):
        tree = et.parse(self.xml)
        root = tree.getroot()
        return root


# class ParseTrips:
#     def __init__(self, root):
#         self.root = root
#     def get_trips(self):


first_q = Query('xmls/RS_Via-3.xml')
root = first_q.get_xml_root()

steps_dict = {'OnwardPricedItinerary': 'start',
              'ReturnPricedItinerary': 'finish', 'Pricing': 'price'}

CONFIG_FLIGHT = \
    {
        'Carrier': {
            'dtype': 'text',
            'attr': {
                'id': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Airline_id'
                }
            },
            'descrip': 'Airline'
        },
        'FlightNumber': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Number_of_flight'
        },
        'Source': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'From'
        },
        'Destination': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'To'
        },
        'DepartureTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_from'
        },
        'ArrivalTimeStamp': {
            'dtype': 'datetime',
            'attr': False,
            'descrip': 'Time_to'
        },
        'Class': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Class'
        },
        'NumberOfStops': {
            'dtype': 'int',
            'attr': False,
            'descrip': 'Count_stops'
        },
        'FareBasis': {
            'dtype': 'id',
            'attr': False,
            'descrip': 'Trip_id'
        },
        'WarningText': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Description'
        },
        'TicketType': {
            'dtype': 'text',
            'attr': False,
            'descrip': 'Type_ticket'
        }

    }
CONFIG_PRICE = \
    {
        'ServiceCharges': {
            'dtype': 'float',
            'attr': {
                'type': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Person_type'
                },
                'ChargeType': {
                    'dtype': 'text',
                    'attr': False,
                    'descrip': 'Rate_type'
                }
            },
            'descrip': 'Charges'
        },
    }


def parse_step(node):
    tag = node.tag
    l_step = list()
    if tag in steps_dict:
        l_step = parse_flights(node, tag)
    return l_step


def get_tag(root, config, attr=None):
    tags = dict()
    for tag in config:
        if attr:
            item = root.attrib
            if len(item) > 1:
                item = {k: item[k] for k in item if k == tag}
        else:
            item = root.find(tag) if root is not None else None
        if config[tag]['attr']:
            tags.update(get_tag(item, config[tag]['attr'], attr=True))
        data = check_dtype(item, config, attr)
        tags.update(data)
    return tags


def check_dtype(item, config, attr):
    d = dict()
    if attr is not None:
        tag = next(iter(item))
        data = item[tag]
    else:
        tag = item.tag
        data = item.text
    if tag in config:
        descrip = config[tag]['descrip']
        dtype = config[tag]['dtype']
        if dtype == 'datetime':
            data = parser.parse(data)
        elif dtype == 'int':
            data = int(data)
        elif dtype == 'float':
            data = float(data)
        elif dtype == 'id':
            data = hash(data.strip())
    d[descrip] = data
    return d


def parse_flights(node, tag):
    trip_flights = list()
    transfer_flight = None
    for cur_flight in range(len(node.findall('.//Flight')) - 1, -1, -1):
        row = get_tag(node.findall('.//Flight')[cur_flight], CONFIG_FLIGHT)
        row['Tag'] = tag
        from_local_t = get_time_by_local(row['From'], row['Time_from'])
        to_local_t = get_time_by_local(row['To'], row['Time_to'])
        if from_local_t['code'] == 200 and to_local_t['code'] == 200:
            row['Time_from_local'] = parser.parse(from_local_t['data'])
            row['Time_to_local'] = parser.parse(to_local_t['data'])
        if cur_flight > 0:
            row['transfer_to'] = transfer_flight
            transfer_flight = row['Number_of_flight']
        else:
            row['transfer_to'] = transfer_flight
        trip_flights.append(row)
    return trip_flights


def parse_trip_price(trip, trip_id):
    trip_prices = list()
    for price in trip.findall('.//Pricing'):
        for single_price in price.findall('ServiceCharges'):
            price_d = dict()
            price_d.update(check_dtype(single_price, CONFIG_PRICE, attr=None))
            price_d['currency'] = price.attrib['currency']
            price_d['Trip_id'] = trip_id
            price_attrib = single_price.attrib
            if price_attrib:
                for k in price_attrib:
                    item = {k: price_attrib[k]}
                    price_d.update(
                        check_dtype(item,
                                    CONFIG_PRICE[single_price.tag]['attr'],
                                    attr=True))
            trip_prices.append(price_d)
    return trip_prices


def get_trips(root):
    if 'Flights' == root.tag:
        return [root]
    flights = list()
    for child in root:
        flights += get_trips(child)
    return flights


t1 = datetime.datetime.now()
trips = get_trips(root)
flights = list()
prices = list()
for trip in trips:
    flight = list()
    for trips_step in list(trip):
        trips_step = parse_step(trips_step)
        if trips_step:
            flight.extend(trips_step)
    if flight:
        flights.extend(flight)
        trip_id = flight[0]['Trip_id']
    price = parse_trip_price(trip, trip_id)
    if price:
        prices.extend(price)
df = pd.DataFrame(data=flights)
# df = df.sort_values(
#     ['Trip_id', 'Tag', 'transfer_to', 'Time_from_local']).reset_index(drop=True)
prices = pd.DataFrame(data=prices)
print(datetime.datetime.now() - t1)

# k = 0
# for i, flight in df.iterrows():
#     if i == 0:
#         trip_id = flight['Trip_id']
#         tag = flight['Tag']
#         start_time = flight['Time_from_local']
#         finish_time = flight['Time_to_local']
#     elif trip_id == flight['Trip_id'] and tag == flight['Tag']:
#         pause = flight['Time_from_local'] - finish_time
#     elif trip_id != flight['Trip_id'] or tag != flight['Tag']:
#         all_time = df.loc[i - 1, 'Time_to_local'] - start_time
#         trip_id = flight['Trip_id']
#         tag = flight['Tag']
#         start_time = flight['Time_from_local']
#         finish_time = flight['Time_to_local']
#         print(all_time)
# print(200)


0:00:00.684030


In [659]:
df[df['Tag'] == 'ReturnPricedItinerary']

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to
2,AirIndia,AI,U,0,None,DEL,995,ReturnPricedItinerary,2018-10-30 20:40:00,2018-10-30 18:10:00,2018-10-30 22:45:00,2018-10-30 21:45:00,DXB,7731301084171169716,E,NaN
3,AirIndia,AI,U,0,None,BKK,333,ReturnPricedItinerary,2018-10-30 08:50:00,2018-10-30 04:50:00,2018-10-30 12:05:00,2018-10-30 09:35:00,DEL,7731301084171169716,E,995.0
6,Malaysia Airlines,MH,N,0,None,KUL,162,ReturnPricedItinerary,2018-10-30 15:15:00,2018-10-30 10:15:00,2018-10-30 18:50:00,2018-10-30 17:50:00,DXB,6604706034725870067,E,NaN
7,Malaysia Airlines,MH,N,0,None,BKK,785,ReturnPricedItinerary,2018-10-30 11:05:00,2018-10-30 07:05:00,2018-10-30 14:15:00,2018-10-30 09:15:00,KUL,6604706034725870067,E,162.0
10,Malaysia Airlines,MH,N,0,None,KUL,162,ReturnPricedItinerary,2018-10-30 15:15:00,2018-10-30 10:15:00,2018-10-30 18:50:00,2018-10-30 17:50:00,DXB,1654381142034801735,E,NaN
11,Malaysia Airlines,MH,N,0,None,BKK,797,ReturnPricedItinerary,2018-10-30 05:55:00,2018-10-30 01:55:00,2018-10-30 09:05:00,2018-10-30 04:05:00,KUL,1654381142034801735,E,162.0
14,Malaysia Airlines,MH,N,0,None,KUL,162,ReturnPricedItinerary,2018-10-30 15:15:00,2018-10-30 10:15:00,2018-10-30 18:50:00,2018-10-30 17:50:00,DXB,-2202666819395507986,E,NaN
15,Malaysia Airlines,MH,N,0,None,BKK,785,ReturnPricedItinerary,2018-10-30 11:05:00,2018-10-30 07:05:00,2018-10-30 14:15:00,2018-10-30 09:15:00,KUL,-2202666819395507986,E,162.0
18,Malaysia Airlines,MH,N,0,None,KUL,162,ReturnPricedItinerary,2018-10-30 15:15:00,2018-10-30 10:15:00,2018-10-30 18:50:00,2018-10-30 17:50:00,DXB,-2808093838591605253,E,NaN
19,Malaysia Airlines,MH,N,0,None,BKK,797,ReturnPricedItinerary,2018-10-30 05:55:00,2018-10-30 01:55:00,2018-10-30 09:05:00,2018-10-30 04:05:00,KUL,-2808093838591605253,E,162.0


In [601]:

df = df.sort_values(['Trip_id','Tag','transfer_to','Time_from_local'])

df

,index,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to
0,254,Pakistan International Airlines,PK,U,0,None,DXB,204,OnwardPricedItinerary,2018-10-27 13:00:00,2018-10-27 12:00:00,2018-10-27 17:00:00,2018-10-27 15:00:00,LHE,-9164081078935422697,E,346.0
1,253,Thai,TG,Y,0,None,LHE,346,OnwardPricedItinerary,2018-10-28 00:05:00,2018-10-27 22:05:00,2018-10-28 06:30:00,2018-10-28 02:30:00,BKK,-9164081078935422697,E,NaN
2,133,China Eastern Airlines,MU,R,0,None,DXB,756,OnwardPricedItinerary,2018-10-27 20:45:00,2018-10-27 19:45:00,2018-10-28 07:15:00,2018-10-28 02:15:00,KMG,-9000761809758366288,E,741.0
3,132,China Eastern Airlines,MU,R,0,None,KMG,741,OnwardPricedItinerary,2018-10-28 14:00:00,2018-10-28 09:00:00,2018-10-28 14:55:00,2018-10-28 10:55:00,BKK,-9000761809758366288,E,NaN
4,284,Emirates,EK,B,0,None,DXB,346,OnwardPricedItinerary,2018-10-27 04:10:00,2018-10-27 03:10:00,2018-10-27 15:15:00,2018-10-27 10:15:00,KUL,-8795564162163048642,E,780.0
5,283,Malaysia Airlines,MH,S,0,None,KUL,780,OnwardPricedItinerary,2018-10-27 18:05:00,2018-10-27 13:05:00,2018-10-27 19:10:00,2018-10-27 15:10:00,BKK,-8795564162163048642,E,NaN
6,330,Qantas,QF,S,0,None,DXB,8354,OnwardPricedItinerary,2018-10-27 03:15:00,2018-10-27 02:15:00,2018-10-27 14:40:00,2018-10-27 09:40:00,SIN,-8596059523334321759,E,414.0
7,329,Thai,TG,Y,0,None,SIN,414,OnwardPricedItinerary,2018-10-27 15:55:00,2018-10-27 10:55:00,2018-10-27 17:15:00,2018-10-27 13:15:00,BKK,-8596059523334321759,E,NaN
8,286,Emirates,EK,B,0,None,DXB,346,OnwardPricedItinerary,2018-10-27 04:10:00,2018-10-27 03:10:00,2018-10-27 15:15:00,2018-10-27 10:15:00,KUL,-8544928968329196853,E,796.0
9,285,Malaysia Airlines,MH,S,0,None,KUL,796,OnwardPricedItinerary,2018-10-27 21:55:00,2018-10-27 16:55:00,2018-10-27 23:00:00,2018-10-27 19:00:00,BKK,-8544928968329196853,E,NaN


In [602]:
task_airports = ['DXB','BKK']
trip_struct = dict()
trips = list()
pauses = list()
for i, flight in df.iterrows():
    if i == 0:
        start_f = flight
    elif trip_id != flight['Trip_id'] or tag != flight['Tag']:
        all_time = df.loc[i - 1, 'Time_to_local'] - start_f['Time_from_local']
        if start_f['From'] in task_airports or flight['To'] in task_airports:
            trip_struct['id'] = start_f['Trip_id']
            trip_struct['from'] = start_f['From']
            trip_struct['to'] = flight['To']
            trips.append(trip_struct)
        start_f = flight
    else:
         pauses.append(flight['Time_from_local'] - start_f['Time_to_local'])
print(trips)



[{'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'from': 'DXB', 'to': 'BKK'}, {'id': 9162412019413242861, 'f

In [473]:
# БЕЗ ПЕРЕСАДОК
df.loc[(df['From'] == 'DXB') & (df['To'] == 'BKK'),'flag'] = 'without_transfer'
df



,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to,flag
61,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,830.0,NaN
60,Qatar Airways,QR,N,0,None,DOH,830,OnwardPricedItinerary,2018-10-27 20:40:00,2018-10-27 20:40:00,2018-10-28 06:55:00,2018-10-28 02:55:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
77,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,NaN
76,Qatar Airways,QR,N,0,None,DOH,832,OnwardPricedItinerary,2018-10-28 08:30:00,2018-10-28 08:30:00,2018-10-28 18:45:00,2018-10-28 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
194,Qatar Airways,QR,M,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,834.0,NaN
193,Qatar Airways,QR,M,0,None,DOH,834,OnwardPricedItinerary,2018-10-28 01:20:00,2018-10-28 01:20:00,2018-10-28 11:35:00,2018-10-28 07:35:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
63,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,836.0,NaN
62,Qatar Airways,QR,N,0,None,DOH,836,OnwardPricedItinerary,2018-10-28 01:55:00,2018-10-28 01:55:00,2018-10-28 12:10:00,2018-10-28 08:10:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN
103,Qatar Airways,QR,S,0,None,DWC,1033,OnwardPricedItinerary,2018-10-27 06:25:00,2018-10-27 05:25:00,2018-10-27 06:30:00,2018-10-27 06:30:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,NaN
102,Qatar Airways,QR,S,0,None,DOH,832,OnwardPricedItinerary,2018-10-27 08:30:00,2018-10-27 08:30:00,2018-10-27 18:45:00,2018-10-27 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,NaN


In [463]:
# количество часов рейс

df['flight_time'] = df.apply(lambda row: row['Time_to_local'] - row['Time_from_local'],axis=1)

df

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to,flight_time
61,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,830.0,01:05:00
60,Qatar Airways,QR,N,0,None,DOH,830,OnwardPricedItinerary,2018-10-27 20:40:00,2018-10-27 20:40:00,2018-10-28 06:55:00,2018-10-28 02:55:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
77,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,01:05:00
76,Qatar Airways,QR,N,0,None,DOH,832,OnwardPricedItinerary,2018-10-28 08:30:00,2018-10-28 08:30:00,2018-10-28 18:45:00,2018-10-28 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
194,Qatar Airways,QR,M,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,834.0,01:05:00
193,Qatar Airways,QR,M,0,None,DOH,834,OnwardPricedItinerary,2018-10-28 01:20:00,2018-10-28 01:20:00,2018-10-28 11:35:00,2018-10-28 07:35:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
63,Qatar Airways,QR,N,0,None,DWC,1031,OnwardPricedItinerary,2018-10-27 19:50:00,2018-10-27 18:50:00,2018-10-27 19:55:00,2018-10-27 19:55:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,836.0,01:05:00
62,Qatar Airways,QR,N,0,None,DOH,836,OnwardPricedItinerary,2018-10-28 01:55:00,2018-10-28 01:55:00,2018-10-28 12:10:00,2018-10-28 08:10:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00
103,Qatar Airways,QR,S,0,None,DWC,1033,OnwardPricedItinerary,2018-10-27 06:25:00,2018-10-27 05:25:00,2018-10-27 06:30:00,2018-10-27 06:30:00,DOH,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,832.0,01:05:00
102,Qatar Airways,QR,S,0,None,DOH,832,OnwardPricedItinerary,2018-10-27 08:30:00,2018-10-27 08:30:00,2018-10-27 18:45:00,2018-10-27 14:45:00,BKK,\n2820303decf751-5511-447a-aeb1-810a6b10ad7d@@...,E,NaN,06:15:00


In [452]:
import io
import requests
import pandas as pd
import redis
import pytz
from tzlocal import get_localzone
from datetime import datetime

r = redis.Redis('localhost')


def search_timezone_by_iata(search_code):
    status = check_iata_spr()
    if status == 200:
        search_code = search_code.upper()
        timezone = r.hget('iata_timezone', search_code)
        if timezone is not None:
            return {'code': 200, 'data': timezone.decode('UTF-8')}
        return {'code': 500, 'msg': 'No data by key in redis'}
    return status


def get_iata_country_spr():
    url = "https://raw.githubusercontent.com/opentraveldata/opentraveldata/master/opentraveldata/optd_por_public.csv"
    try:
        s = requests.get(url).content
    except ConnectionError:
        return {'code': 500, 'msg': 'ConnectionError'}
    country_map = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='^')
    iata_dict = country_map.loc[:,
                ['iata_code', 'country_code', 'timezone']].drop_duplicates(
        subset=['iata_code']).to_dict(orient='records')
    iata_dict = {k['iata_code']: k['timezone'] for k in iata_dict}
    r.hmset('iata_timezone', iata_dict)
    check_r = r.exists('iata_timezone')
    if check_r:
        return {'code': 200}
    else:
        return {'code': 500, 'msg': 'No data in redis'}


def check_iata_spr():
    if not r.exists('iata_timezone'):
        load_spr = get_iata_country_spr()
        if 'code' in load_spr:
            if load_spr['code'] == 200:
                return 200
        return load_spr
    else:
        return 200


def get_time_by_local(iata_code, dt):
    timezone = search_timezone_by_iata(iata_code)
    if timezone['code'] == 200:
        timezone = pytz.timezone(timezone['data'])
        timezone_dt = timezone.localize(dt)
        local_zone = pytz.timezone(str(get_localzone()))
        local_dt = timezone_dt.astimezone(local_zone)
        local_dt = datetime.strftime(local_dt, '%Y-%m-%d %T')
        return {'code': 200, 'data': local_dt}
    return timezone


# t1 = datetime.now()
print(get_time_by_local('HKT', datetime(2002, 10, 27, 6, 0, 0)))
# print(datetime.now() - t1)


{'code': 200, 'data': '2002-10-27 02:00:00'}


In [608]:
a = prices.groupby(['Trip_id']).agg({'Charges':'sum'}).reset_index()

In [613]:
a.loc[a['Trip_id'] == -9164081078935422697,'Charges'].values[0]

14364.0

In [670]:
prices

,Charges,Person_type,Rate_type,Trip_id,currency
0,117.0,SingleAdult,BaseFare,7731301084171169716,SGD
1,429.8,SingleAdult,AirlineTaxes,7731301084171169716,SGD
2,546.8,SingleAdult,TotalAmount,7731301084171169716,SGD
3,563.0,SingleAdult,BaseFare,6604706034725870067,SGD
4,60.8,SingleAdult,AirlineTaxes,6604706034725870067,SGD
5,623.8,SingleAdult,TotalAmount,6604706034725870067,SGD
6,563.0,SingleAdult,BaseFare,1654381142034801735,SGD
7,60.8,SingleAdult,AirlineTaxes,1654381142034801735,SGD
8,623.8,SingleAdult,TotalAmount,1654381142034801735,SGD
9,563.0,SingleAdult,BaseFare,-2202666819395507986,SGD


In [669]:
cols = prices.columns
agg_prices = prices.groupby(['Trip_id'])[cols].agg(
        {'Charges': 'sum'}).reset_index()


Trip_id Charges              
                         Charges       Trip_id
0   -9213379257109739127  1690.0 -2.764014e+19
1   -9054865084340367886  1690.0 -2.716460e+19
2   -8937095715014460741  1690.0 -2.681129e+19
3   -8711292512407693359  1690.0 -2.613388e+19
4   -8490957345042403860  1247.6 -2.547287e+19
5   -8416082543706578001  2759.6 -2.524825e+19
6   -8323714215048537852  2169.6 -2.497114e+19
7   -8319872800650679198  1690.0 -2.495962e+19
8   -8311845236971656863  1384.4 -2.493554e+19
9   -8301790811643776548  1690.0 -2.490537e+19
10  -8229308993164590055  2465.6 -2.468793e+19
11  -8067657735888823376  1690.0 -2.420297e+19
12  -8066413944861112737  1690.0 -2.419924e+19
13  -7916174615542256261  1690.0 -2.374852e+19
14  -7748386189260112676  2169.6 -2.324516e+19
15  -7729897635321939792  1294.8 -2.318969e+19
16  -7565107428397401882  1690.0 -2.269532e+19
17  -7521973894665325647  2169.6 -2.256592e+19
18  -7436132683582906322  1690.0 -2.230840e+19
19  -7364366768833879119  1690.0 -2.209310e+19
20  -7354331700732590984  1690.0 -2.206300e+19
21  -7263963095959522908  1512.0 -2.179189e+19
22  -7178051920575677863  1776.8 -2.153416e+19
23  -6958049803410603741  2465.6 -2.087415e+19
24  -6805845175641418206  1690.0 -2.041754e+19
25  -6723056496446243223  1512.0 -2.016917e+19
26  -6589063433575967078  2169.6 -1.976719e+19
27  -6558730359108107071  2465.6 -1.967619e+19
28  -6470743586989038049  1690.0 -1.941223e+19
29  -6319446220386323250  2465.6 -1.895834e+19
..                   ...     ...           ...
170  6318033401872722183  2169.6  1.895410e+19
171  6491220333250651831  1247.6  1.947366e+19
172  6550139084285538613  1690.0  1.965042e+19
173  6565310268602705114  1690.0  1.969593e+19
174  6604706034725870067  1247.6  1.981412e+19
175  6813537222851895658  1690.0  2.044061e+19
176  6916749566611736184  1690.0  2.075025e+19
177  7009111666121328909  1690.0  2.102733e+19
178  7054996654153639782  1790.0  2.116499e+19
179  7122145020012266879  1690.0  2.136644e+19
180  7239325547162778628  1690.0  2.171798e+19
181  7261274010053316316  1690.0  2.178382e+19
182  7459767028227241274  1690.0  2.237930e+19
183  7465661578260098848  1690.0  2.239698e+19
184  7477535459124556594  1690.0  2.243261e+19
185  7705010927235631490  1690.0  2.311503e+19
186  7731301084171169716  1093.6  2.319390e+19
187  7859591550238260678  1690.0  2.357877e+19
188  7931834463216616494  1690.0  2.379550e+19
189  8087145364291603997  1690.0  2.426144e+19
190  8100451743193727345  1690.0  2.430136e+19
191  8284005464049728126  1512.0  2.485202e+19
192  8317420523072514068  1690.0  2.495226e+19
193  8324196439469629875  1690.0  2.497259e+19
194  8478529082137081286  1930.8  2.543559e+19
195  8481176174117454334  2465.6  2.544353e+19
196  8645372418474404068  1247.6  2.593612e+19
197  8749830515715506000  2465.6  2.624949e+19
198  8756150535461768742  2169.6  2.626845e+19
199  9073306751606742312  2465.6  2.721992e+19

[200 rows x 3 columns]

In [673]:
agg_prices.loc[agg_prices['Trip_id']==-8889177882129451753,:]


Empty DataFrame
Columns: [(Trip_id, ), (Charges, Charges), (Charges, Trip_id)]
Index: []

In [642]:
str(local_zone)
Europe/Moscow

'Europe/Moscow'

In [634]:
url = "https://raw.githubusercontent.com/opentraveldata/opentraveldata/master/opentraveldata/optd_por_public.csv"
s = requests.get(url).content
country_map = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='^')


In [640]:
a = country_map[['country_code','timezone']].drop_duplicates()

In [647]:
a[a['timezone'] == str(local_zone)]

,country_code,timezone
34,RU,Europe/Moscow


In [668]:
df

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to
0,AirIndia,AI,G,0,None,DEL,332,OnwardPricedItinerary,2018-10-22 13:50:00,2018-10-22 11:20:00,2018-10-22 19:35:00,2018-10-22 15:35:00,BKK,7731301084171169716,E,NaN
1,AirIndia,AI,G,0,None,DXB,996,OnwardPricedItinerary,2018-10-22 00:05:00,2018-10-21 23:05:00,2018-10-22 04:45:00,2018-10-22 02:15:00,DEL,7731301084171169716,E,332.0
2,AirIndia,AI,U,0,None,DEL,995,ReturnPricedItinerary,2018-10-30 20:40:00,2018-10-30 18:10:00,2018-10-30 22:45:00,2018-10-30 21:45:00,DXB,7731301084171169716,E,NaN
3,AirIndia,AI,U,0,None,BKK,333,ReturnPricedItinerary,2018-10-30 08:50:00,2018-10-30 04:50:00,2018-10-30 12:05:00,2018-10-30 09:35:00,DEL,7731301084171169716,E,995.0
4,Malaysia Airlines,MH,L,0,None,KUL,5860,OnwardPricedItinerary,2018-10-23 13:20:00,2018-10-23 08:20:00,2018-10-23 14:30:00,2018-10-23 10:30:00,BKK,6604706034725870067,E,NaN
5,Malaysia Airlines,MH,N,0,None,DXB,163,OnwardPricedItinerary,2018-10-22 19:35:00,2018-10-22 18:35:00,2018-10-23 07:05:00,2018-10-23 02:05:00,KUL,6604706034725870067,E,5860.0
6,Malaysia Airlines,MH,N,0,None,KUL,162,ReturnPricedItinerary,2018-10-30 15:15:00,2018-10-30 10:15:00,2018-10-30 18:50:00,2018-10-30 17:50:00,DXB,6604706034725870067,E,NaN
7,Malaysia Airlines,MH,N,0,None,BKK,785,ReturnPricedItinerary,2018-10-30 11:05:00,2018-10-30 07:05:00,2018-10-30 14:15:00,2018-10-30 09:15:00,KUL,6604706034725870067,E,162.0
8,Malaysia Airlines,MH,L,0,None,KUL,5860,OnwardPricedItinerary,2018-10-23 13:20:00,2018-10-23 08:20:00,2018-10-23 14:30:00,2018-10-23 10:30:00,BKK,1654381142034801735,E,NaN
9,Malaysia Airlines,MH,N,0,None,DXB,163,OnwardPricedItinerary,2018-10-22 19:35:00,2018-10-22 18:35:00,2018-10-23 07:05:00,2018-10-23 02:05:00,KUL,1654381142034801735,E,5860.0


In [671]:
df.groupby(['Trip_id']).apply(lambda x: x.to_json(orient='records'))

Trip_id
-9213379257109739127    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-9054865084340367886    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-8937095715014460741    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-8711292512407693359    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-8490957345042403860    [{"Airline":"Malaysia Airlines","Airline_id":"...
-8416082543706578001    [{"Airline":"Emirates","Airline_id":"EK","Clas...
-8323714215048537852    [{"Airline":"Emirates","Airline_id":"EK","Clas...
-8319872800650679198    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-8311845236971656863    [{"Airline":"China Southern Airlines","Airline...
-8301790811643776548    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-8229308993164590055    [{"Airline":"Emirates","Airline_id":"EK","Clas...
-8067657735888823376    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-8066413944861112737    [{"Airline":"Qatar Airways","Airline_id":"QR",...
-7916174615542256261    [{"Air

In [679]:
df[df['Trip_id'] == 8844558837137708891]

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to


In [678]:
df

,Airline,Airline_id,Class,Count_stops,Description,From,Number_of_flight,Tag,Time_from,Time_from_local,Time_to,Time_to_local,To,Trip_id,Type_ticket,transfer_to
0,AirIndia,AI,G,0,None,DEL,332,OnwardPricedItinerary,2018-10-22 13:50:00,2018-10-22 11:20:00,2018-10-22 19:35:00,2018-10-22 15:35:00,BKK,7731301084171169716,E,NaN
1,AirIndia,AI,G,0,None,DXB,996,OnwardPricedItinerary,2018-10-22 00:05:00,2018-10-21 23:05:00,2018-10-22 04:45:00,2018-10-22 02:15:00,DEL,7731301084171169716,E,332.0
2,AirIndia,AI,U,0,None,DEL,995,ReturnPricedItinerary,2018-10-30 20:40:00,2018-10-30 18:10:00,2018-10-30 22:45:00,2018-10-30 21:45:00,DXB,7731301084171169716,E,NaN
3,AirIndia,AI,U,0,None,BKK,333,ReturnPricedItinerary,2018-10-30 08:50:00,2018-10-30 04:50:00,2018-10-30 12:05:00,2018-10-30 09:35:00,DEL,7731301084171169716,E,995.0
4,Malaysia Airlines,MH,L,0,None,KUL,5860,OnwardPricedItinerary,2018-10-23 13:20:00,2018-10-23 08:20:00,2018-10-23 14:30:00,2018-10-23 10:30:00,BKK,6604706034725870067,E,NaN
5,Malaysia Airlines,MH,N,0,None,DXB,163,OnwardPricedItinerary,2018-10-22 19:35:00,2018-10-22 18:35:00,2018-10-23 07:05:00,2018-10-23 02:05:00,KUL,6604706034725870067,E,5860.0
6,Malaysia Airlines,MH,N,0,None,KUL,162,ReturnPricedItinerary,2018-10-30 15:15:00,2018-10-30 10:15:00,2018-10-30 18:50:00,2018-10-30 17:50:00,DXB,6604706034725870067,E,NaN
7,Malaysia Airlines,MH,N,0,None,BKK,785,ReturnPricedItinerary,2018-10-30 11:05:00,2018-10-30 07:05:00,2018-10-30 14:15:00,2018-10-30 09:15:00,KUL,6604706034725870067,E,162.0
8,Malaysia Airlines,MH,L,0,None,KUL,5860,OnwardPricedItinerary,2018-10-23 13:20:00,2018-10-23 08:20:00,2018-10-23 14:30:00,2018-10-23 10:30:00,BKK,1654381142034801735,E,NaN
9,Malaysia Airlines,MH,N,0,None,DXB,163,OnwardPricedItinerary,2018-10-22 19:35:00,2018-10-22 18:35:00,2018-10-23 07:05:00,2018-10-23 02:05:00,KUL,1654381142034801735,E,5860.0
